# mutclust 알고리즘 돌려보기

#2026-02-06

---

## Load Data

먼저 필수 패키지와 mutclust 함수들을 불러온다.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from src.utils import *
from src.arg_parser import *
from src.mlib import *

다음으로 핫스팟 탐지에 사용할 SARS-CoV-2 바이러스 돌연변이 데이터를 불러온다.

In [ ]:
data = pd.read_csv("GISAID_total.csv")
print(f"Data shape: {data.shape}")
print(data.head())

데이터를 확인해보면 row는 29903개로써 29903개 염기 위치에 대한 돌연변이 데이터이고, column은 5개로써 각각 Position, Frequency, Percentage, Entropy, 계산된 H-중요도 정보다.

## Run Mutclust

데이터를 불러왔으니 mutclust를 실행해보자.

In [ ]:
tag = "test1"
info = set_env(input="GISAID_total.csv", output=tag)
Path(tag).mkdir(parents=True, exist_ok=True)
init(data, info)

tag는 첫번째 테스트라는 의미로 "test1"을 넣어줬다. 결과는 "test1" 디렉토리에 저장한다.

이제 실행해본다. mutclust는 핫스팟 클러스터링 알고리즘으로, 1) 클러스터의 중심 돌연변이 탐색 2) 클러스터 반경 조정 2개 단계로 수행된다.


In [ ]:
# Find candidate core mutations (CCMs)
total_mutInfo, ccm_index = get_candidate_core_mutations(data, info, tag, 1)
print(f"CCM count: {len(ccm_index)}")

실행 결과, 1990개 CCM이 클러스터의 중심으로 선택되었다.

다음으로 local clustering을 실행해보자.

In [ ]:
# Find mutation hotspots
clusters = dynaclust(total_mutInfo, ccm_index, info, tag, 1)
print(f"Cluster count: {len(clusters)}")

mutclust 알고리즘으로 총 477개 돌연변이 핫스팟이 식별되었다.

## Results

In [ ]:
# Load CCM results
ccms = pd.read_csv(f"{tag}/ccm_{tag}.tsv", sep="\t")
print(f"CCMs: {len(ccms)}")
print(ccms.head())

In [ ]:
# Load hotspot results
hotspots = pd.read_csv(f"{tag}/clusters_{tag}.txt", sep="\t")
print(f"Hotspots: {len(hotspots)}")
print(hotspots.head())

결과를 확인해보면 1) H-중요도로 산정했을때 총 1990개 CCM이 클러스터 중심 돌연변이로 선택되었고 2) 밀도 기반 Local clustering 결과 477개 돌연변이 핫스팟이 식별되었다. 

식별된 돌연변이 핫스팟을 bedgraph로 시각화해보면 아래와 같다.

In [ ]:
# Visualization
output_path = f"{tag}/bedgraph_{tag}.png"
make_bedgraph(data, hotspots, output_path)
display_image(output_path)